In [1]:
import av
import numpy as np
import torch
from torchvision import transforms
from torch.utils.data import Dataset, random_split
from transformers import (
    AutoProcessor,
    BitsAndBytesConfig,
    VideoLlavaForConditionalGeneration,
    Trainer,
    TrainingArguments
)
from peft import get_peft_model, LoraConfig, TaskType, prepare_model_for_kbit_training
import os
from typing import Tuple
import pandas as pd

In [2]:
# Constants
MODEL_ID = "LanguageBind/Video-LLaVA-7B-hf"
MODEL_NAME = MODEL_ID.split("/")[-1]

# File/directory
VIDEO_DIR = "/scratch/as18464/raw_videos"
CSV_FILE = "valid_clips.csv"
CACHE_DIR = "cache/"
DATASET_SIZE = 125
TRAIN_VAL_SPLIT = 0.8

# Model constants
BATCH_SIZE = 4
MAX_LENGTH = 128  # Fixed sequence length for text
NUM_FRAMES = 16   # Fixed number of frames
IMAGE_SIZE = 224  # Fixed image size

# Training hyperparameters
LEARNING_RATE = 1e-4
WEIGHT_DECAY = 0.05
NUM_EPOCHS = 20

In [3]:
def read_video_pyav(container, indices):
    '''
    Decode the video with PyAV decoder.
    Args:
        container (`av.container.input.InputContainer`): PyAV container.
        indices (`List[int]`): List of frame indices to decode.
    Returns:
        result (np.ndarray): np array of decoded frames of shape (num_frames, height, width, 3).
    '''
    frames = []
    container.seek(0)
    start_index = indices[0]
    end_index = indices[-1]

    resize_transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])

    for i, frame in enumerate(container.decode(video=0)):
        if i > end_index:
            break
        if i >= start_index and i in indices:
            # Convert to numpy array in RGB format
            frame_array = frame.to_ndarray(format="rgb24")
            # Apply resize transform and convert back to numpy
            resized_frame = resize_transform(frame_array).numpy()
            # Convert from CxHxW to HxWxC format and scale back to 0-255 range
            resized_frame = (resized_frame.transpose(1, 2, 0) * 255).astype(np.uint8)
            frames.append(resized_frame)
    
    return np.stack(frames)

def get_frames(video_path: str, num_frames: int = 8) -> np.ndarray:
    """
    Extract frames from video with consistent sampling
    Args:
        video_path (str): Path to video file
        num_frames (int): Number of frames to extract
    Returns:
        np.ndarray: Array of frames with shape (num_frames, height, width, 3)
    """
    container = av.open(video_path)
    
    # Get video stream
    stream = container.streams.video[0]
    total_frames = stream.frames
    fps = stream.average_rate
    
    # Calculate indices to sample
    indices = np.linspace(0, total_frames - 1, num_frames, dtype=int)
    
    # Read frames at calculated indices
    frames = read_video_pyav(container, indices)
    
    # Ensure we got exactly num_frames
    if len(frames) < num_frames:
        # If we got fewer frames, duplicate the last frame
        last_frame = frames[-1]
        while len(frames) < num_frames:
            frames = np.concatenate([frames, last_frame[np.newaxis, ...]], axis=0)
    elif len(frames) > num_frames:
        # If we got more frames, take the first num_frames
        frames = frames[:num_frames]
    
    container.close()
    return frames


class VideoDataset(Dataset):
    def __init__(self, video_dir: str, annotations: pd.DataFrame, processor, num_frames: int = 16):
        self.video_dir = video_dir
        self.annotations = annotations
        self.num_frames = num_frames
        self.processor = processor
        self.system_prompt = ("Analyze the American Sign Language (ASL) signs in this video and "
                            "translate them into clear, natural English. Consider the sequence of "
                            "signs as a complete message, and provide an accurate translation that "
                            "captures the full meaning. Respond with only the English translation, "
                            "without descriptions of the signs themselves.")
        
        print(f"Created dataset split with {len(self.annotations)} entries")
    
    def __len__(self) -> int:
        return len(self.annotations)
    
    def __getitem__(self, idx: int) -> dict:
        row = self.annotations.iloc[idx]
        video_id = str(row['SENTENCE_NAME']).strip()
        sentence = str(row['SENTENCE']).strip()
        
        video_path = os.path.join(self.video_dir, f"{video_id}.mp4")
        if not os.path.isfile(video_path):
            raise FileNotFoundError(f"Video file '{video_path}' not found.")
        
        # Get video frames using the provided functions
        frames = get_frames(video_path, self.num_frames)
        prompt = f"USER: {self.system_prompt}\n<video>\nASSISTANT: {sentence}"
        
        # Process the frames and text with fixed sizes
        inputs = self.processor(
            text=prompt,
            videos=[frames],  # frames is already in the correct format from get_frames
            padding="max_length",  # Always pad to max_length
            truncation=True,
            max_length=MAX_LENGTH,
            return_tensors="pt"
        )
        
        # Create labels from input_ids
        labels = inputs["input_ids"].clone()
        labels[labels == self.processor.tokenizer.pad_token_id] = -100
        
        # Mask everything before and including "ASSISTANT:"
        assistant_start = None
        for j in range(len(inputs["input_ids"][0])):
            if self.processor.tokenizer.decode(inputs["input_ids"][0][j:j+4]) == "ASSISTANT:":
                assistant_start = j
                break
        
        if assistant_start is not None:
            labels[0, :assistant_start+4] = -100
        
        # Return tensors with consistent sizes
        return {
            "input_ids": inputs["input_ids"].squeeze(0),
            "attention_mask": inputs["attention_mask"].squeeze(0),
            "pixel_values_videos": inputs["pixel_values_videos"].squeeze(0),
            "labels": labels.squeeze(0)
        }

def create_train_val_datasets(video_dir: str, csv_file: str, processor, num_frames: int = 16):
    # Read the full dataset
    full_df = pd.read_csv(csv_file, sep=',').head(DATASET_SIZE).reset_index(drop=True)
    
    # Calculate split sizes
    train_size = int(len(full_df) * TRAIN_VAL_SPLIT)
    val_size = len(full_df) - train_size
    
    # Randomly shuffle the dataframe
    shuffled_df = full_df.sample(frac=1, random_state=42).reset_index(drop=True)
    
    # Split the dataframe
    train_df = shuffled_df.iloc[:train_size]
    val_df = shuffled_df.iloc[train_size:]
    
    # Create dataset objects
    train_dataset = VideoDataset(video_dir, train_df, processor, num_frames)
    val_dataset = VideoDataset(video_dir, val_df, processor, num_frames)
    
    return train_dataset, val_dataset

In [ ]:
def main():
    # Set up device and processor
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    processor = AutoProcessor.from_pretrained(MODEL_ID)
    processor.tokenizer.padding_side = "right"
    processor.image_processor.do_rescale = False

    processor.patch_size = 14  # Standard patch size for ViT-L

    # Create train and validation datasets
    train_dataset, val_dataset = create_train_val_datasets(
        video_dir=VIDEO_DIR,
        csv_file=CSV_FILE,
        processor=processor,
        num_frames=NUM_FRAMES
    )

    # Initialize model with quantization
    quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16
    )

    model = VideoLlavaForConditionalGeneration.from_pretrained(
        MODEL_ID,
        torch_dtype=torch.float16,
        device_map="auto",
        cache_dir=CACHE_DIR,
        quantization_config=quantization_config
    )

    # Prepare model for k-bit training and configure LoRA
    model = prepare_model_for_kbit_training(model)
    peft_config = LoraConfig(
        r=8,
        lora_alpha=32,
        target_modules=["q_proj", "v_proj", "k_proj", "o_proj", 
                       "gate_proj", "up_proj", "down_proj"],
        lora_dropout=0.1,
        bias="none",
        task_type=TaskType.CAUSAL_LM
    )
    model = get_peft_model(model, peft_config)

    # Configure training arguments
    training_args = TrainingArguments(
        output_dir="output",
        num_train_epochs=NUM_EPOCHS,
        per_device_train_batch_size=BATCH_SIZE,
        per_device_eval_batch_size=BATCH_SIZE,
        gradient_accumulation_steps=1,
        learning_rate=LEARNING_RATE,
        weight_decay=WEIGHT_DECAY,
        fp16=True,
        logging_dir="logs",
        logging_steps=10,
        save_strategy="epoch",
        evaluation_strategy="epoch",
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
        greater_is_better=False,
        remove_unused_columns=False,
        ddp_find_unused_parameters=True,
        dataloader_num_workers=0,
    )

    # Initialize trainer without custom collator
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=None,
    )

    # Start training
    trainer.train()

if __name__ == "__main__":
    main()

Created dataset split with 100 entries
Created dataset split with 25 entries


The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/ext3/miniforge3/lib/python3.12/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/ext3/miniforge3/lib/python3.12/site-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/ext3/miniforge3/lib/python3.12/site-packages/torch/utils/checkpoint.py:92: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/ext3/miniforge3/lib/python3.12/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast

Epoch,Training Loss,Validation Loss
1,2.873900,2.848508
2,2.353100,2.800480
3,1.465600,2.957610
4,0.817000,3.402862
5,0.512600,3.722869
6,0.383200,3.797446
7,0.338000,3.968263
8,0.333300,3.969143
9,0.281400,4.190957
10,0.196800,4.542109


/ext3/miniforge3/lib/python3.12/site-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/ext3/miniforge3/lib/python3.12/site-packages/torch/utils/checkpoint.py:92: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/ext3/miniforge3/lib/python3.12/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/ext3/miniforge3/lib

In [4]:
from peft import PeftModel, PeftConfig

def load_model_from_checkpoint(
    base_model_id="LanguageBind/Video-LLaVA-7B-hf",
    checkpoint_dir="output/checkpoint-500",
    device="cuda"
):  
    # Initialize processor with required attributes
    processor = AutoProcessor.from_pretrained(base_model_id)
    processor.tokenizer.padding_side = "right"
    processor.image_processor.do_rescale = False
    processor.patch_size = 14

    print("Loading base model...")
    # Load the base model
    model = VideoLlavaForConditionalGeneration.from_pretrained(
        base_model_id,
        torch_dtype=torch.float16,
        device_map=device
    )
    
    print("Loading adapter weights...")
    # Load the adapter weights (LoRA) from safetensors format
    model = PeftModel.from_pretrained(
        model,
        checkpoint_dir,
        device_map=device,
        torch_dtype=torch.float16,
        is_trainable=False  # Set to False for inference
    )
    
    print("Setting model to evaluation mode...")
    # Set model to evaluation mode
    model.eval()
    
    return model, processor

In [5]:
BASE_MODEL_ID = "LanguageBind/Video-LLaVA-7B-hf"
CHECKPOINT_DIR = "output/checkpoint-500"  # Your checkpoint directory

In [6]:
# Load model
device = "cuda" if torch.cuda.is_available() else "cpu"
try:
    model, processor = load_model_from_checkpoint(
        base_model_id=BASE_MODEL_ID,
        checkpoint_dir=CHECKPOINT_DIR,
        device=device
    )
    print("Model loaded successfully!")
except Exception as e:
    print(f"Error loading model: {str(e)}")
    raise

Loading base model...


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Loading adapter weights...
Setting model to evaluation mode...
Model loaded successfully!


In [49]:
def simple_inference(model, processor, video_path):
    """
    Simple inference function for Video-LLaVA model
    
    Args:
        model: The loaded Video-LLaVA model
        processor: The Video-LLaVA processor
        video_path: Path to the video file
    
    Returns:
        str: The translated text
    """
    # Set model to evaluation mode
    model.eval()
    
    # Get video frames
    frames = get_frames(video_path, num_frames=16)
    
    # Create prompt
    prompt = "Translate the sign language in this video to English text.\n<video>\nASSISTANT:"
    
    # Process inputs
    inputs = processor(
        text=prompt,
        videos=[frames],
        return_tensors="pt",
        padding=True,
        truncation=True
    )
    
    # Move inputs to GPU if available
    inputs = {k: v.to(model.device) for k, v in inputs.items() if isinstance(v, torch.Tensor)}
    
    # Generate text
    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            pixel_values_videos=inputs["pixel_values_videos"],
            max_new_tokens=50,
            num_beams=3,
            do_sample=False
        )
    
    # Decode the generated text
    translated_text = processor.batch_decode(outputs, skip_special_tokens=True)[0]
    
    # Clean up the response
    if "ASSISTANT:" in translated_text:
        translated_text = translated_text.split("ASSISTANT:")[-1].strip()
    
    return translated_text

In [52]:
video_path = '/scratch/as18464/raw_videos/-0FA6OhxZSE_9-5-rgb_front.mp4'
simple_inference(model, processor, video_path)

'And����--------�--------�--------�--------�--------����--------�--------�--------�----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------'